In [21]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import numpy as np
import requests
import pandas as pd
import re
from selenium.webdriver.support.ui import Select
from dataclasses import dataclass, field


In [57]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)



Mozilla/5.0 (Windows; U; Windows NT 5.1; it-IT) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.3 Safari/533.19.4


## Scrap urls

In [35]:
urls = [
    {'url':'https://janoubco.com/Chandeliers_and_chandeliers/%D9%85%D9%88%D8%AF%D8%B1%D9%86?limit=100','cat1':'ثريات ونجف', 'cat2': 'مودرن'},
    {'url':'https://janoubco.com/Chandeliers_and_chandeliers/%D9%83%D9%84%D8%A7%D8%B3%D9%8A%D9%83?limit=100','cat1':'ثريات ونجف', 'cat2': 'كلاسيك'},
    {'url':'https://janoubco.com/Chandeliers_and_chandeliers/%D8%B9%D9%84%D8%A7%D9%82%D9%8A?limit=100','cat1':'ثريات ونجف', 'cat2': 'علاقي'},
    {'url':'https://janoubco.com/%D9%85%D9%81%D8%A7%D8%AA%D9%8A%D8%AD_%D9%88%D8%A7%D9%81%D9%8A%D8%A7%D8%B4/%D8%A7%D8%A8%D9%8A%D8%B6_%D8%B3%D8%AA%D9%8A%D9%84?limit=100','cat1':'مفاتيح وافياش', 'cat2': 'ابيض ستيل'},
    {'url':'https://janoubco.com/%D9%85%D9%81%D8%A7%D8%AA%D9%8A%D8%AD_%D9%88%D8%A7%D9%81%D9%8A%D8%A7%D8%B4/%D8%A3%D8%A8%D9%8A%D8%B6?limit=100','cat1':'مفاتيح وافياش', 'cat2': 'أبيض'},
    {'url':'https://janoubco.com/%D8%A5%D9%86%D8%A7%D8%B1%D8%A9_%D8%AF%D8%A7%D8%AE%D9%84%D9%8A%D8%A9/%D8%B4%D8%B1%D9%8A%D8%B7_%D9%84%D9%8A%D8%AF?limit=100','cat1':'إنارة داخلية', 'cat2': 'شريط ليد زينة'},
    {'url':'https://janoubco.com/%D8%A5%D9%86%D8%A7%D8%B1%D8%A9_%D8%AF%D8%A7%D8%AE%D9%84%D9%8A%D8%A9/%D9%84%D9%85%D8%A8%D8%A9_%D9%83%D8%B1%D9%88%D9%8A%D8%A9?limit=100','cat1':'إنارة داخلية', 'cat2': 'لمبة كروية'},
]

In [36]:
df = pd.read_excel('janoubco_model_urls1.xlsx')

for i, url1 in enumerate(urls):
    print('Count: ', i)
    url =  url1['url']
    cat1 = url1['cat1']
    cat2 = url1['cat2']
    driver.get(url)
    text = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
    soup = BeautifulSoup(text, 'html.parser')
    products = soup.find('div', {'class': 'main-products product-grid'}).find_all('div', {'class': 'name'})
    print(len(products))
    list_products = [pro.find('a')['href']for pro in products]
    data = []
    for product in list_products:
        data.append(
            {
                'url': product,
                'cat1': cat1,
                'cat2': cat2,
            }
        )
    df1 = pd.DataFrame(data)
    df = pd.concat([df, df1], ignore_index=True)
    df.to_excel('Janoubco_url_product.xlsx')
   

Count:  0
100
Count:  1
53
Count:  2
100
Count:  3
14
Count:  4
42
Count:  5
14
Count:  6
54


In [51]:
text = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(text, 'html.parser')


## SCrap product


In [55]:
toto = pd.read_excel('Janoubco_url_product.xlsx') 

list_urls = []

for ind, row in toto.iterrows():
    list_urls.append({
        'url': row['url'],
        'cat1': row['cat1'],
        'cat2': row['cat2'],
    })

In [65]:
df = pd.read_excel('janoubco_model_product.xlsx')


for i, url1 in enumerate(list_urls):
    url = url1['url']
    cat1 = url1['cat1']
    cat2 = url1['cat2']
    print('Count: ', i)
    print('URL: ', url)
    driver.get(url)
    text = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
    soup = BeautifulSoup(text, 'html.parser')
    time.sleep(1)
    name = soup.find('div', {'class': 'title page-title'}).text.strip()
    price = soup.find('div', {'class': 'product-price'}).text.replace('ريال', '').strip()
    is_in_stock = soup.find('li', {'class': 'product-stock in-stock'}).find('span').text.strip()

    sku = soup.find('li', {'class': 'product-sku'}).find('span').text.strip()
    model = soup.find('li', {'class': 'product-model'}).find('span').text.strip()
    description = soup.find('div', {'class': 'panel-body block-content'}).text.strip()
    images = soup.find('div', {'class': 'product-left'}).find_all('img')
    len(images)
    list_images = [ img['src'] for img in images if '550x550' in img['src']]
    base_images = list_images[0]
    ms_brand = soup.find('div', {'class': 'brand-image product-manufacturer'}).find('span').text.strip()
    print('ms_brand')
    try:
        add_images = ','.join(list_images[1:])
    except:
        add_images = ''
    data = {
        'link_url': url,
        'name': name,
        'sku': sku,
        'price': price,
        'is_in_stock': is_in_stock,
        'model': model,
        'ms_brand': ms_brand,
        
        'cat1': cat1,
        'cat2': cat2,
        'description': description,
        'base_image': base_images,
        'add_images': add_images,
        
        
        
    }
    df1 = pd.DataFrame([data])
    df = pd.concat([df, df1], ignore_index=True)
    df.to_excel('janoubco-Update_product.xlsx')


Count:  0
URL:  https://janoubco.com/Chandeliers_and_chandeliers/مودرن?product_id=1335&limit=100
ms_brand
Count:  1
URL:  https://janoubco.com/Chandeliers_and_chandeliers/مودرن?product_id=1360&limit=100
ms_brand
Count:  2
URL:  https://janoubco.com/Chandeliers_and_chandeliers/مودرن?product_id=743&limit=100
ms_brand
Count:  3
URL:  https://janoubco.com/Chandeliers_and_chandeliers/مودرن?product_id=746&limit=100
ms_brand
Count:  4
URL:  https://janoubco.com/Chandeliers_and_chandeliers/مودرن?product_id=739&limit=100
ms_brand
Count:  5
URL:  https://janoubco.com/Chandeliers_and_chandeliers/مودرن?product_id=738&limit=100
ms_brand
Count:  6
URL:  https://janoubco.com/Chandeliers_and_chandeliers/مودرن?product_id=751&limit=100
ms_brand
Count:  7
URL:  https://janoubco.com/Chandeliers_and_chandeliers/مودرن?product_id=748&limit=100
ms_brand
Count:  8
URL:  https://janoubco.com/Chandeliers_and_chandeliers/مودرن?product_id=67&limit=100
ms_brand
Count:  9
URL:  https://janoubco.com/Chandeliers_and_c

In [62]:
images = soup.find('div', {'class': 'product-left'}).find_all('img')
len(images)
list_images = [ img['src'] for img in images if '550x550' in img['src']]
list_images

['https://janoubco.com/image/cache/catalog/photos/product5/HA-A4359-5-550x550.png',
 'https://janoubco.com/image/cache/catalog/photos/product3/HA-A4359-5-550x550h.jpg']